In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import numpy as np
import scipy.special
from cosmoprimo import Cosmology, PowerSpectrumBAOFilter
import argparse
import multiprocessing
from utils_paths import path_template
from utils import cosmology, redshift_distributions
from utils_template import PowerSpectrumMultipoles, CorrelationFunctionMultipoles, WThetaCalculator

# Arguments
class Args:
    def __init__(self):
        self.cosmology_template = "planck"
        self.nz_flag = "fid"
        self.include_wiggles = "y"
args = Args()
args.include_wiggles = '' if args.include_wiggles == 'y' else '_nowiggles'

# 1. Path for saving the template
path = path_template(include_wiggles=args.include_wiggles, nz_flag=args.nz_flag, cosmology_template=args.cosmology_template)()
os.makedirs(path, exist_ok=True)

# 2. Numerical resolution of the calculation
# Nz, Nk, Nmu, Nr = 10**3, 2*10**5, 5*10**4, 5*10**4
Nz, Nk, Nmu, Nr = 10**3, 10**4, 10**4, 10**4

# 3. Values of theta
theta = 10**np.linspace(np.log10(0.001), np.log10(179.5), 10**2) * np.pi / 180

# 4. Redshift distributions
nz_instance = redshift_distributions(args.nz_flag)

# 5. Cosmology
params = cosmology(args.cosmology_template)
cosmo = Cosmology(
    h=params.h,
    Omega_cdm=params.Omega_m - params.Omega_b - params.Omega_nu_massive,
    Omega_b=params.Omega_b,
    sigma8=params.sigma_8,
    n_s=params.n_s,
    Omega_ncdm=params.Omega_nu_massive,
    N_eff=3.046,
    engine='class'
)

# 6. Calculation of pk_ell
pk_calculator = PowerSpectrumMultipoles(
    cosmo=cosmo,
    include_wiggles=args.include_wiggles,
    nz_instance=nz_instance,
    Nk=Nk,
    Nmu=Nmu,
    path=path,
)
with multiprocessing.Pool(nz_instance.nbins) as pool:
    pk_ell_dict = pool.map(pk_calculator.compute_pk_ell, range(nz_instance.nbins))

# 7. Calculation of xi_ell
xi_calculator = CorrelationFunctionMultipoles(
    power_spectrum_multipoles=pk_calculator,
    Nr=Nr,
)
with multiprocessing.Pool(nz_instance.nbins) as pool:
    xi_ell_dict = pool.map(xi_calculator.compute_xi_ell, range(nz_instance.nbins))

# 8. Calculation of w(theta)
calculator = WThetaCalculator(correlation_function_multipoles=xi_calculator, Nz=Nz)
for bin_z in range(nz_instance.nbins):
    calculator.compute_and_save_wtheta(bin_z)


Saving output to: wtheta_template/nz_fid/wtheta_planck
Using fid n(z), which has 6 redshift bins
Initialized cosmology: planck
0 - Computing Pk_ell...1 - Computing Pk_ell...2 - Computing Pk_ell...3 - Computing Pk_ell...4 - Computing Pk_ell...5 - Computing Pk_ell...





2 - Pk_ell computed!
0 - Pk_ell computed!
1 - Pk_ell computed!
4 - Pk_ell computed!
3 - Pk_ell computed!
5 - Pk_ell computed!
0 - Computing xi_ell...1 - Computing xi_ell...2 - Computing xi_ell...3 - Computing xi_ell...4 - Computing xi_ell...5 - Computing xi_ell...



2 - Loading precomputed Pk_ell data...1 - Loading precomputed Pk_ell data...3 - Loading precomputed Pk_ell data...
0 - Loading precomputed Pk_ell data...


4 - Loading precomputed Pk_ell data...


5 - Loading precomputed Pk_ell data...
0 - 10%
2 - 10%
1 - 10%
5 - 10%
4 - 10%
3 - 10%
0 - 20%
2 - 20%
1 - 20%
5 - 20%
3 - 20%
4 - 20%
0 - 30%
2 - 30%
1 - 30%
3 - 30%
5 - 30%
4 - 30%
0 - 40%
2 - 40%
3 - 40%
1 - 40%
5 - 40%
4 - 40%
0 - 50%
2 - 50%
3 - 50%
1 - 50%
5